In [37]:
import json,os,sys,logging,pprint

In [38]:
nGramDict=json.load(open('/home/yewenhe0904/Developing/ds-capstone-project/ignoredFiles/Sample-Dict/news-nGramDict.json'))

In [39]:
nGramDict['_n']['W7']

{'_n': {'W1': {'_r': ['W3017']},
  'W108': {'_r': ['W31']},
  'W11': {},
  'W168': {'_r': ['W1103', 'W1358', 'W715']},
  'W2': {'_r': ['W3', 'W168']},
  'W246': {'_r': ['W247']},
  'W35': {'_r': ['W102', 'W1319']},
  'W522': {'_r': ['W4917', 'W18823']},
  'W577': {'_r': ['W31']},
  'W66521': {'_r': ['W1']},
  'W7048': {'_r': ['W129']},
  'W8': {'_r': ['W9', 'W855']},
  'W9444': {'_r': ['W11']}},
 '_r': ['W35', 'W168', 'W8', 'W2', 'W522']}

In [61]:
test_input='the news report'

In [62]:
test_tokens=test_input.lower().split()

In [63]:
input_tokens=test_tokens[-2:]
input_tokens

['news', 'report']

In [64]:
def predictCore(input_tokens,output_list):
    #end condition
    print (input_tokens)
    if len(input_tokens)<1:
        output_list+=(nGramDict['_r'])
    else:
        # probe if in ngram record
        nGram_flag=True
        i=0
        p_Dict=nGramDict['_n']
        while nGram_flag:
            print(i)
            if i==len(input_tokens):
                break
            if (input_tokens[i] in p_Dict):
                r_wids=p_Dict[input_tokens[i]]['_r']
                if ('_n' in p_Dict[input_tokens[i]]):
                    p_Dict=p_Dict[input_tokens[i]]['_n']
                    i+=1
                else:
                    if i<len(input_tokens)-1:
                        nGram_flag=False
                    break
            else:
                nGram_flag=False
                break
        # if true
        if nGram_flag:
            output_list+=r_wids   
        # else false
        else:
            del input_tokens[0]
            predictCore(input_tokens,output_list)


        
        
        

In [65]:
def predictNgram(input_tokens):
    output_list=[]
    output_wlist=[]
    input_idtokens=[]
    for w in input_tokens:
        w.strip()
        if w in nGramDict['_word2id']['_word']:
            input_idtokens.append(nGramDict['_word2id']['_word'][w])
        else:
            input_idtokens.append('NA')
    predictCore(input_idtokens,output_list)
    for wid in output_list:
        output_wlist.append(nGramDict['_id2word']['_id'][wid])
    return output_wlist

In [66]:
predictNgram(input_tokens)

['W342', 'W2484']
0
1


['relied', '<EOS>']